In [6]:
!pip install mplfinance

In [7]:
import yfinance as yf
import pandas as pd
import talib
import matplotlib.pyplot as plt
import mplfinance as mpf

In [25]:
data['Open'].values.reshape(60,)

1

In [27]:

data = yf.download("SPY", start="2025-01-01", end="2025-04-01")

morning_star = talib.CDLMORNINGSTAR(data['Open'].values.reshape(60,), data['High'].values.reshape(60,), data['Low'].values.reshape(60,), data['Close'].values.reshape(60,))

engulfing = talib.CDLENGULFING(data['Open'].values.reshape(60,), data['High'].values.reshape(60,), data['Low'].values.reshape(60,), data['Close'].values.reshape(60,))

data['Morning Star'] = morning_star
data['Engulfing'] = engulfing

engulfing_days = data[data['Engulfing'] != 0]

print(engulfing_days)

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume  \
Ticker             SPY         SPY         SPY         SPY       SPY   
Date                                                                   
2025-01-31  600.015015  608.130616  599.247305  605.677972  66671500   
2025-02-07  598.968201  606.306112  598.250328  605.069841  50788500   
2025-03-03  582.019165  595.548473  578.160777  594.391918  74249200   
2025-03-05  581.311279  583.125828  571.361231  574.960389  71230500   
2025-03-07  574.192688  575.658311  563.933572  569.187785  81158800   

Price      Morning Star Engulfing  
Ticker                             
Date                               
2025-01-31            0      -100  
2025-02-07            0      -100  
2025-03-03            0      -100  
2025-03-05            0       100  
2025-03-07            0       100  


In [33]:
data = yf.download(ticker, start=start_date, end=end_date, progress=False)

In [37]:
data.swaplevel(axis=1)

Ticker             QQQ                                               
Price            Close        High         Low        Open     Volume
Date                                                                 
2025-03-03  496.309845  512.276025  492.845011  510.758299   44587200
2025-03-04  494.812103  502.989911  487.013735  493.444148   67875400
2025-03-05  501.262482  502.880065  490.528489  495.461136   46323700
2025-03-06  487.473053  497.837571  485.476031  492.954868   57574300
2025-03-07  491.057709  492.545480  479.814466  486.434598   54714700
2025-03-10  472.026093  483.249346  467.962146  482.730131   76668000
2025-03-11  470.897766  478.097021  466.314605  471.646649   68760800
2025-03-12  476.209869  480.343689  471.087503  478.446524   46971100
2025-03-13  467.642609  475.570788  465.735449  475.540834   46563300
2025-03-14  478.945770  479.844424  472.415500  473.024608   43563000
2025-03-17  482.051117  485.116553  477.088516  478.895819   41744400
2025-03-18  473.833405  478.895832  471.706571  478.696148   39072300
2025-03-19  480.173950  484.607311  474.162895  476.100008   40430300
2025-03-20  478.546356  483.828457  475.470925  476.169895   36780500
2025-03-21  480.123993  480.892835  472.205809  473.374049   42234900
2025-03-24  490.660004  491.510010  484.399994  487.739990   34567200
2025-03-25  493.459991  493.619995  490.420013  491.390015   26035200
2025-03-26  484.380005  493.160004  482.820007  492.299988   34627100
2025-03-27  481.619995  486.579987  480.250000  482.410004   33469800
2025-03-28  468.940002  480.519989  468.049988  479.809998   46363800
2025-03-31  468.920013  469.859985  457.329987  461.920013   53000300
2025-04-01  472.700012  473.630005  464.420013  467.299988   41156200
2025-04-02  476.149994  479.559998  465.859985  466.119995   49894500
2025-04-03  450.660004  460.070007  450.140015  456.440002   70456300
2025-04-04  422.670013  440.369995  422.670013  438.140015  117088400
2025-04-07  423.690002  443.140015  402.390015  408.660004  161557000
2025-04-08  416.059998  443.140015  409.790009  438.160004  101248100
2025-04-09  466.000000  467.829987  415.429993  415.570007  142876900
2025-04-10  446.179993  455.589996  432.630005  453.559998  108384100
2025-04-11  454.399994  455.790009  441.329987  444.649994   52483800

In [38]:
def download_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    data.swaplevel(axis=1)
    # Drop the 'Close' column and use the 'Adj Close'
    # data = data.drop(columns=['Close'])
    # # Check if the necessary columns exist
    # required_columns = ['Open', 'High', 'Low', 'Volume']
    # if not all(col in data.columns for col in required_columns):
    #     print(f"Missing columns: {', '.join([col for col in required_columns if col not in data.columns])}")
    # data.columns = range(len(data.columns))  
    # # Rename columns to desired names
    # data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    # data = data.dropna(subset=['Close', 'High', 'Low', 'Open', 'Volume'])
    # # Keep only required columns and rename 'Adj Close' to 'Close' for mpl finance
    # data = data[['Close', 'Open', 'High', 'Low', 'Volume']]

    # Ensure the index is of datetime type
    data.index = pd.to_datetime(data.index)
    
    # Drop rows with missing values in the relevant columns
    data = data.dropna()
    
    return data

In [39]:
def detect_candlestick_patterns(df):
    # Drop rows with missing values
    #df = df.dropna(subset=['Open', 'High', 'Low', 'Adj Close'])

    # Check if there are enough rows to calculate patterns
    if len(df) < 5:
        print("Not enough data to detect patterns.")
        return df

    # Convert DataFrame columns to numpy arrays
    open_prices = df['Open'].values.reshape(-len(df),)
    high_prices = df['High'].values.reshape(-len(df),)
    low_prices = df['Low'].values.reshape(-len(df),)
    close_prices = df['Close'].values.reshape(-len(df),)  # Use 'Adj Close' instead of 'Close'
    
    # Ensure all arrays have the same length
    if not (len(open_prices) == len(high_prices) == len(low_prices) == len(close_prices)):
        print("Input arrays have different lengths.")
        return df
    
    # Candlestick pattern detection
    patterns = {
        'Engulfing': talib.CDLENGULFING(open_prices, high_prices, low_prices, close_prices),
        'Morning Star': talib.CDLMORNINGSTAR(open_prices, high_prices, low_prices, close_prices, penetration=0.3),
        'Hammer': talib.CDLHAMMER(open_prices, high_prices, low_prices, close_prices),
        'Inverted Hammer': talib.CDLINVERTEDHAMMER(open_prices, high_prices, low_prices, close_prices),
        'Piercing Pattern': talib.CDLPIERCING(open_prices, high_prices, low_prices, close_prices),
        'Three White Soldiers': talib.CDL3WHITESOLDIERS(open_prices, high_prices, low_prices, close_prices),
        'Rising Three Methods': talib.CDLRISEFALL3METHODS(open_prices, high_prices, low_prices, close_prices),
    }

    # Add detected patterns to the DataFrame
    for pattern_name, pattern in patterns.items():
        df[pattern_name] = pattern
    return df


# Filter for selected patterns
def filter_patterns(df):
    detected_patterns = df[(df['Engulfing'] != 0) |
                           (df['Morning Star'] != 0) |
                           (df['Hammer'] != 0) |
                           (df['Inverted Hammer'] != 0) |
                           (df['Piercing Pattern'] != 0) |
                           (df['Three White Soldiers'] != 0) |
                           (df['Rising Three Methods'] != 0)]
    return detected_patterns

In [40]:
def plot_candlestick_chart(df, ticker):
    # Calculate pivot point, support, and resistance levels
    #df['Pivot Point'] = (df['High'] + df['Low'] + df['Close']) / 3
   # df['Support 1'] = (2 * df['Pivot Point']) - df['High']
    #df['Resistance 1'] = (2 * df['Pivot Point']) - df['Low']
    # Calculate 8-day EMA
    df['8-Day EMA'] = df['Close'].ewm(span=8, adjust=False).mean()
    # Add pivot points, support/resistance, and 8-EMA
    ap_lines = [
       # mpf.make_addplot(df['Pivot Point'], color='black', linestyle='--', width=1.5, label='Pivot Point'),
       # mpf.make_addplot(df['Support 1'], color='red', linestyle='--', width=2.5, alpha=0.6, label='Support 1'),
       # mpf.make_addplot(df['Resistance 1'], color='green', linestyle='--', width=2.5, alpha=0.6, label='Resistance 1'),
        mpf.make_addplot(df['8-Day EMA'], color='blue', linestyle='-', width=2, label='8-Day EMA')
    ]

    # Plot the candlestick chart
    mpf.plot(
        df,
        type='candle',
        style='charles',
        addplot=ap_lines,
        title=f'{ticker} Candlestick Chart',
        ylabel='Price',
        volume=True,
        figsize=(16, 9) 
    )

In [43]:
tickers = [ 'TQQQ']  
start_date = '2025-03-01'
end_date = '2025-04-14'

for ticker in tickers:
    df = download_data(ticker, start_date, end_date)
    print(df)
    df_with_patterns = detect_candlestick_patterns(df)
    detected_patterns = filter_patterns(df_with_patterns)
    detected_patterns.index = detected_patterns.index.strftime('%Y-%m-%d')
    
    print(f"\nDetected candlestick patterns for {ticker}:\n")
    pattern_columns = detected_patterns.columns[5:]  # Adjusted to the relevant pattern columns
    for idx, row in detected_patterns.iterrows():
        non_zero_patterns = row[pattern_columns][row[pattern_columns] != 0]
        if not non_zero_patterns.empty:
            print(f"Date: {idx}, Patterns: {non_zero_patterns.to_dict()}\n")
    
    # Plot the candlestick chart for each ticker
    plot_candlestick_chart(df, ticker)

Price           Close       High        Low       Open     Volume
Ticker           TQQQ       TQQQ       TQQQ       TQQQ       TQQQ
Date                                                             
2025-03-03  69.873856  76.823354  68.238680  76.145354   91945800
2025-03-04  69.106117  72.555938  65.815829  68.557740  130836900
2025-03-05  71.798180  72.476180  67.291471  69.345417  100633300
2025-03-06  65.845734  70.292613  64.978290  68.188819  122822700
2025-03-07  67.211708  67.809942  62.705000  65.367144  135177300
2025-03-10  59.634064  64.001181  57.709741  63.811737  156135800
2025-03-11  58.936123  61.668061  57.201240  59.225271  139489300
2025-03-12  60.950180  62.475678  58.985974  61.757795  141918600
2025-03-13  57.620007  60.671003  56.872213  60.621151  121227300
2025-03-14  61.707943  62.046943  59.324974  59.534355   99108500
2025-03-17  62.904415  64.090913  60.970123  61.697975   90150400
2025-03-18  59.733768  61.598267  58.836414  61.588298   87826200
2025-03-19

ValueError: Data for column "Open" must be ALL float or int.